# 학습기법 적용하기
- FashionMNIST를 이용한 MLP 학습

In [1]:
''' 1. Module Import '''
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.0.0+cpu  Device: cpu


In [3]:
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
''' 3. FashionMNIST 데이터 다운로드 (Train set, Test set 분리하기) '''
train_dataset = datasets.FashionMNIST(root = "../data/FashionMNIST",
                               train = True,
                               download = True,
                               transform = transforms.ToTensor())

test_dataset = datasets.FashionMNIST(root = "../data/FashionMNIST",
                              train = False,
                              transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

In [5]:
''' 4. 데이터 확인하기 (1) '''
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [ ]:
''' 5. 데이터 확인하기 (2) '''
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
    plt.title('Class: ' + str(y_train[i].item()))

## Dropout /  Batch Normalization 적용하기

In [ ]:
''' 6. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
        #dropout 퍼센트 정하기
        self.dropout_prob = 0.5
        
        #Batch Normalization
        self.batch_norm1 = nn.BatchNorm1d(512) # 첫 번째 Layer의 Output = 512
        self.batch_norm2 = nn.BatchNorm1d(256) # 두 번째 Layer의 Output = 256

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        
        x = F.dropout(x, training = self.training, p = self.dropout_prob) #함수 결과값에 드롭아웃 적용
        
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        
        x = F.dropout(x, training = self.training, p = self.dropout_prob) #함수 결과값에 드롭아웃 적용
        
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x

## Initalization
```python
# Xaiver Initalization
torch.nn.init.xavier_uniform_(tensor, gain=1.0)
torch.nn.init.xavier_normal_(tensor, gain=1.0)

# He Initalization
torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
```
[참고 : torch.nn.init](https://pytorch.org/docs/stable/nn.init.html)

## Optimizer 설정
```python
# SGD
torch.optim.SGD(model.parameters(), lr = 0.01)
    
# SGD (+momentum)
torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

# Adam
torch.optim.Adam(model.parameters(), lr = 0.01)
```
[참고 : torch.optim](https://pytorch.org/docs/stable/optim.html)

In [ ]:
''' 7. Optimizer, Objective Function 설정하기 '''
# Initialization

# init 모듈 : weight, bias 등 초깃값으로 설정되는 요소에 대한 모듈
import torch.nn.init as init
def weight_init(m): # weight 초기화
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data) #He Initialization (앞서 ReLU 함수 적용했기 때문에)

model = Net().to(DEVICE)
model.apply(weight_init)

#Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

print(model)

In [ ]:
''' 8. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
''' 9. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
testloss = []
testaccuracy = []

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
    testloss.append(test_loss)
    testaccuracy.append(test_accuracy)

In [ ]:
# 결과 시각화
plt.subplot(121)
plt.plot(testloss)
plt.title('Loss')

plt.subplot(122)
plt.plot(testaccuracy)
plt.title('Accuracy')

# 
plt.suptitle('Dropout : 0.5 / He Initialization / Adam / Batch Norm',  position = (0.5, 1.0+0.05))

plt.tight_layout()
plt.show()